In [ ]:
import os
import time
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1028"

import torch
torch.cuda.empty_cache()

In [ ]:
import shutil

# delete db
try:
    shutil.rmtree("/home/youse/apperception/eva/evadb_data", ignore_errors=True)
except Exception:
    print("Dir does not exist")
print("deleting db")

In [ ]:
%pip install --quiet "evadb[vision,notebook]"
import evadb
cursor = evadb.connect().cursor()
"""
Issues Encountered:
cant have negative numbers in multidimensional arrays
cant have camel case words
cant have float attributes
cant change column types of udf without restarting db and deleting
query takes infinitley long when UDF in 
inner join does not work
"""

In [ ]:
### Set up Yolo UDF
cursor.query("""
            CREATE UDF IF NOT EXISTS Yolo
            TYPE  ultralytics
            'model' 'yolov8m.pt';
    """).df() 

In [ ]:
### Set up Monodepth UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS MonodepthDetection
            IMPL'/home/youse/apperception/eva/udfs/monodepth_detection.py';
    """).df()

In [ ]:
### Set up Location UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS LocationDetection
            IMPL'/home/youse/apperception/eva/udfs/location_detection.py';
    """).df()

In [ ]:
### Set up Q1 Query UDF
cursor.query(""" 
        CREATE UDF IF NOT EXISTS QE1
        IMPL'/home/youse/apperception/eva/udfs/QE1.py';
""").df()

### Set up Q2 Query UDF
cursor.query(""" 
        CREATE UDF IF NOT EXISTS QE2
        IMPL'/home/youse/apperception/eva/udfs/QE2.py';
""").df()

### Set up Q3 Query UDF
cursor.query(""" 
        CREATE UDF IF NOT EXISTS QE3
        IMPL'/home/youse/apperception/eva/udfs/QE3.py';
""").df()

### Set up Q4 Query UDF
cursor.query(""" 
        CREATE UDF IF NOT EXISTS QE4
        IMPL'/home/youse/apperception/eva/udfs/QE4.py';
""").df()

### Set up SameVideo UDF
cursor.query(""" 
        CREATE UDF IF NOT EXISTS SameVideo
        IMPL'/home/youse/apperception/eva/udfs/same_video.py';
""").df()

In [ ]:
### Set up SameVideo UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS SameVideo
            IMPL'/home/youse/apperception/eva/udfs/same_video.py';
    """).df()

In [ ]:
# Certain attributes are made TEXTs due to issues Eva has with negative numbers
cursor.query("DROP TABLE IF EXISTS CameraConfigs;").df()
cursor.create_table("CameraConfigs", if_not_exists=True, columns="""
                cameraid TEXT(15),
                framenum INTEGER,
                cameratranslation NDARRAY FLOAT32(ANYDIM),
                camerarotation TEXT(100),
                cameraintrinsic NDARRAY FLOAT32(ANYDIM),
                egoheading TEXT(15),
                filename TEXT(30)
            """).df()


In [ ]:
### Load Data
from apperception.database import database
import pandas as pd

cursor.query("DROP TABLE IF EXISTS ObjectDetectionVideos;").df()

with open("scene-names.txt", 'r') as f:
    for sceneNumber in ["0757"]:
        sceneNumber = sceneNumber.strip()
        # Load videos
        video_name = f"boston-seaport-scene-{sceneNumber}-CAM_FRONT.mp4"
        scene = f"scene-{sceneNumber}-CAM_FRONT"
        video_path = "/data/processed/full-dataset/trainval/videos/"
        cursor.load(file_regex=video_path + video_name, format="VIDEO", table_name='ObjectDetectionVideos').df()

        # Add camera configs
        result = database.execute(f"SELECT cameraId, ROW_NUMBER() OVER (Order by frameNum) AS RowNumber, cameraTranslation, cameraRotation, cameraIntrinsic, egoHeading, filename FROM Cameras WHERE cameraId = '{scene}'")
        df = pd.DataFrame()
        for r in result:
            cameraId, frameNum, cameraTranslation, cameraRotation, cameraIntrinsic, egoHeading, filename = r
            cameraTranslation = list(cameraTranslation)
            # FrameNums in Eva are zero-indexed, so we subtract one before inserting
            cursor.query(f"""INSERT INTO CameraConfigs (cameraid, framenum, cameratranslation, camerarotation, cameraintrinsic, egoheading, filename) VALUES
                                        ('{cameraId}', {frameNum - 1}, {cameraTranslation}, '{cameraRotation}', {cameraIntrinsic}, '{egoHeading}', '{filename}');""").df()

        

In [ ]:
### Run query 1
start = time.time()

res1 = cursor.query("""
                SELECT framenum, id, cameraid, filename, name, egoheading, cameratranslation, LocationDetection(Yolo(data), MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic), QE1(LocationDetection(Yolo(data), MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic), cameratranslation, egoheading).queryresult
                    FROM ObjectDetectionVideos JOIN CameraConfigs ON (id = framenum AND SameVideo(name, cameraid).issame)
    """).df()
res1 = res1[res1["qe1.queryresult"]]
end = time.time()
print("Query 1 Time:", format(end-start))

In [ ]:
### Run query 2
start = time.time()

res2 = cursor.query("""
                SELECT framenum, id, cameraid, filename, name, egoheading, cameratranslation, LocationDetection(Yolo(data), MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic), QE2(LocationDetection(Yolo(data), MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic), cameratranslation, egoheading).queryresult
                    FROM ObjectDetectionVideos JOIN CameraConfigs ON (id = framenum AND SameVideo(name, cameraid).issame)
    """).df()
res2 = res2[res2["qe2.queryresult"]]

end = time.time()
print("Query 2 Time:", format(end-start))

In [ ]:
### Run query 3
start = time.time()

res3 = cursor.query("""
                SELECT framenum, id, cameraid, filename, name, egoheading, cameratranslation, LocationDetection(Yolo(data), MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic), QE3(LocationDetection(Yolo(data), MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic), cameratranslation, egoheading).queryresult
                    FROM ObjectDetectionVideos JOIN CameraConfigs ON (id = framenum AND SameVideo(name, cameraid).issame)
    """).df()
res3 = res3[res3["qe3.queryresult"]]

end = time.time()
print("Query 3 Time:", format(end-start))

In [ ]:
### Run query 4
start = time.time()

res4 = cursor.query("""
                SELECT framenum, id, cameraid, filename, name, egoheading, cameratranslation, LocationDetection(Yolo(data), MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic), QE4(LocationDetection(Yolo(data), MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic), cameratranslation, egoheading).queryresult
                    FROM ObjectDetectionVideos JOIN CameraConfigs ON (id = framenum AND SameVideo(name, cameraid).issame)
    """).df()
res4= res4[res4["qe4.queryresult"]]

end = time.time()
print("Query 4 Time:", format(end-start))

In [ ]:
res3

In [ ]:
import cv2
import matplotlib.pyplot as plt


def displayFrame(row):
    prefix = "/data/raw/full-dataset/trainval/"
    # itemIds = ['scene-0757-CAM_FRONT_obj_44', 'scene-0757-CAM_FRONT_obj_23']
    frame_im = cv2.imread(prefix + row['cameraconfigs.filename'])
    plt.imshow(frame_im)


displayFrame(res3.iloc[1])


In [ ]:
res1